# Tema 6: Prompt Engineering

## Ejercicio 1 - Diseño comparativo de prompts

Diseña tres prompts distintos (zero-shot, one-shot y few-shot) para clasificar noticias en {economía, salud, tecnología}.

Evalúa teóricamente cuál será más robusto ante ambigüedad y por qué.

### Zero-shot

```
Classify the topic of the news into {economy, health, technology}. Text: '...'. Topic:
```

### One-shot

```
Example:
Text: 'The hospital improved patient care using new AI systems.'
Topic: health

Now classify:
Text: '...'
Topic:
```

### Few-shot

```
Examples:
Text: 'Stock markets fell after the policy announcement.'
Topic: economy
Text: 'A new vaccine has shown promising results.'
Topic: health
Text: 'The new chip reduces energy consumption.'
Topic: technology

Now classify:
Text: '...'
Topic:
```

### Evaluación

**Few-shot es más robusto** porque fija patrones semánticos y reduce ambigüedad. Al proporcionar múltiples ejemplos de cada categoría, el modelo establece un "anclaje" que le ayuda a desambiguar casos difíciles (p. ej., una noticia sobre IA aplicada a la salud).

## Ejercicio 2 - Descomposición de tareas complejas

Se desea analizar reseñas de productos para:
1. Detectar sentimiento.
2. Extraer conceptos mencionados.
3. Generar un resumen breve.

Diseña una estrategia basada en múltiples prompts encadenados.

Justifica el orden y formato de salida de cada paso.

### Paso 1 - Sentimiento

```
Classify the sentiment as {positive, negative, neutral}. Text: '...'. Sentiment:
```

Se ejecuta primero porque es la tarea más sencilla y su resultado condiciona la interpretación posterior.

### Paso 2 - Extracción de conceptos

```
Extract concepts mentioned in the text. Return a list. Text: '...'. Aspects:
```

Se ejecuta después del sentimiento porque los conceptos extraídos se pueden contextualizar con la polaridad ya detectada.

### Paso 3 - Resumen

```
Summarize the review in one sentence, mentioning the sentiment and key aspects.
```

El resumen se genera al final, integrando la información de los pasos anteriores (sentimiento + conceptos clave).

### Justificación

Separar las tareas en un pipeline multi-prompt mejora:

- **Control**: cada paso se puede evaluar y depurar por separado.
- **Evaluación**: se pueden medir métricas específicas para cada sub-tarea.
- **Reducción de errores acumulados**: un fallo en una tarea no contamina las demás si se detecta a tiempo.

## Ejercicio 3 - Control de formato estructurado

Diseña un prompt que extraiga entidades PERSONA, LUGAR y FECHA del texto dado y valide la salida en JSON.

Incluye restricciones para evitar texto libre o comentarios adicionales.

### Prompt ejemplo

```
Actúa como experto en PLN.
Extrae entidades PERSONA, LUGAR y FECHA.
Devuelve únicamente JSON válido.
No incluyas explicaciones.

Texto: "..."

Formato:
{
  "PERSONA": [],
  "LUGAR": [],
  "FECHA": []
}
```

### Análisis

Las restricciones clave del prompt son:

- **"Devuelve únicamente JSON válido"**: fuerza formato estructurado.
- **"No incluyas explicaciones"**: evita texto libre fuera del JSON.
- **Proporcionar la plantilla JSON**: da un esquema explícito que el modelo debe seguir.

Esto garantiza una salida procesable por código sin necesidad de post-procesamiento adicional.

## Ejercicio 4 - Detección y mitigación de alucinaciones

Propón un prompt para responder preguntas basadas SOLO en un texto proporcionado.

Incluye mecanismos para que el modelo indique cuando no tiene información suficiente.

### Prompt ejemplo

```
Answer ONLY using the information in the provided text.
If the answer is not explicitly present, respond exactly:
"Information not available."

Text: "..."
Question: "..."
Answer:
```

### Análisis

Este prompt introduce un **mecanismo de abstención** para reducir alucinaciones:

- **"Answer ONLY using the information in the provided text"**: restringe la fuente de información al contexto dado.
- **Respuesta por defecto ("Information not available.")**: el modelo tiene una salida explícita para cuando no puede responder, en lugar de inventar información.
- Esto es especialmente útil en dominios críticos (médico, legal, financiero) donde una alucinación puede tener consecuencias graves.

## Ejercicio 5 - Prompt Engineering vs Prompt Mining

Explica cómo automatizarías la búsqueda de un prompt óptimo para clasificación temática.

Describe el proceso, métricas y criterios de selección.

### Proceso de Prompt Mining

El proceso de automatización sigue estos pasos:

1. **Generar múltiples variantes de prompts**: crear variaciones sistemáticas (cambios en formato, instrucciones, ejemplos, orden de las categorías, etc.).
2. **Evaluarlos con un dataset etiquetado**: ejecutar cada variante sobre un conjunto de evaluación con etiquetas ground-truth.
3. **Medir métricas**: F1-score, consistencia (misma entrada produce misma salida) y longitud de respuesta.
4. **Seleccionar el prompt con mejor rendimiento**: elegir la variante que maximice las métricas objetivo.

**Ventaja**: automatiza la optimización frente al diseño manual, que depende de la intuición del diseñador y puede dejar sin explorar variantes superiores.

## Ejercicio 6 - Chain-of-Thought aplicado

Diseña un prompt con Chain-of-Thought para resolver un problema de razonamiento numérico.

### Prompt ejemplo

```
Solve step by step.
Q: A store had 45 items, sold 17, and received 8 more. How many items now?
A:
```

### Análisis

**Ventajas de Chain-of-Thought (CoT):**
- Útil en tareas de razonamiento donde descomponer el problema mejora la precisión.
- Permite verificar cada paso intermedio.
- Mejora la transparencia del razonamiento del modelo.

**Limitaciones:**
- No usar cuando se requiera latencia baja o respuesta corta.
- Aumenta el número de tokens generados (y por tanto el coste).
- En tareas triviales, puede ser innecesario y añadir complejidad sin beneficio.